In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\adars\\Desktop\\AI--Powered-Medical-Chatbot-for-Intelligent-Healthcare-Assistance\\research'

In [3]:
import os 
os.chdir("../")
%pwd

'c:\\Users\\adars\\Desktop\\AI--Powered-Medical-Chatbot-for-Intelligent-Healthcare-Assistance'

In [4]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [5]:
#Extract the data from the pdf file 
#data is the directory of the pdf file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [6]:
extracted_data = load_pdf_file(data="Data/")

In [7]:
#split the data into chunks 
def text_split (extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of text_chunks =",len(text_chunks))

length of text_chunks = 5860


In [9]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [10]:
#download embeddings from hugging face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

C:\Users\adars\AppData\Local\Temp\ipykernel_36944\2527910566.py:3: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\adars\Desktop\AI--Powered-Medical-Chatbot-for-Intelligent-Healthcare-Assistance\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
query_result = embeddings.embed_query("Hello World")
print("Length=",len(query_result))

Length= 384


In [13]:
from dotenv import load_dotenv
import os 
load_dotenv()
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
#print(PINECONE_API_KEY)

In [14]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os 
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-cdymvxs.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [15]:
import os 
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY

In [16]:
from langchain_pinecone import PineconeVectorStore
#embed each chunk and upsert the embeddings into your pinecone index
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [17]:
from langchain_pinecone import PineconeVectorStore
docsearch =PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [18]:
docsearch

In [19]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [22]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='c2880c93-b110-4cc1-b7dd-357dc93a7d45', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine. Vol. 1. 2nd ed.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='8bb5675b-54a5-45f4-a3a9-c124fab39b1f', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine. Vol. 1. 2nd ed.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ

In [ ]:
from langchain_openai import OpenAI
llm =OpenAI(temperature=0.4, max_tokens=500)